In [1]:
import pandas as pd
import numpy as np
import os
import sys
import statsmodels.api as sm
print(os.getcwd())
print(pd.__version__)
print(sys.version)

C:\Users\xcy99\Desktop\mypythonscripts
1.0.5
3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]


In [2]:
xls = pd.ExcelFile(r'C:\Users\xcy99\Desktop\OneDrive - The Chinese University of Hong Kong\AAAAAAAAA_CUHK\FINA6112FC investment and portfolio analysis\case\AQR.xlsx')
df1 = pd.read_excel(xls, 'Return',index_col=0, header=2)
df2 = pd.read_excel(xls, 'Portfolio',index_col=0, header=3)

df2.rename(columns={'Mkt-RF': 'Excess_Market'}, inplace=True)
df2.head(10)

,Excess_Market,Unnamed: 2,H,L,HML,Unnamed: 6,S,B,SMB,Unnamed: 10,U,D,UMD
Year,,,,,,,,,,,,,
1927,30.29,NaN,35.095,38.725,-3.630,NaN,31.376667,34.370000,-2.993333,NaN,38.270,15.390,22.880
1928,35.51,NaN,33.685,38.830,-5.145,NaN,37.933333,34.506667,3.426667,NaN,50.310,22.805,27.505
1929,-19.76,NaN,-20.610,-32.530,11.920,NaN,-37.636667,-7.846667,-29.790000,NaN,-21.145,-42.410,21.265
1930,-31.24,NaN,-44.345,-30.850,-13.495,NaN,-37.556667,-33.060000,-4.496667,NaN,-23.185,-49.605,26.420
1931,-45.44,NaN,-54.790,-38.860,-15.930,NaN,-47.240000,-51.470000,4.230000,NaN,-35.325,-59.605,24.280
1932,-9.43,NaN,-0.765,-6.305,5.540,NaN,-3.820000,-9.450000,5.630000,NaN,-14.080,5.435,-19.515
1933,57.20,NaN,120.345,105.195,15.150,NaN,136.943333,82.876667,54.066667,NaN,109.530,90.005,19.525
1934,4.09,NaN,-6.910,22.450,-29.360,NaN,20.650000,-4.660000,25.310000,NaN,21.115,2.965,18.150
1935,44.70,NaN,52.345,39.115,13.230,NaN,55.346667,46.703333,8.643333,NaN,68.880,47.705,21.175


In [3]:
# change first row to all str. need to use astype rather than apply
df1.columns = df1.columns.astype(str)

# problem 1 analyze the returns
df1['Decile Return'] = df1['High']-df1['Low'] #break to 10 groups
df1['Quintile Return'] = (df1['9']+df1['High'])/2-(df1['Low']+df1['2'])/2 #break to 5 groups
df1['Median Return'] = (df1['6']+df1['7']+df1['8']+df1['9']+df1['High'])/5-(df1['Low']+df1['2']+df1['3']+df1['4']+df1['5'])/5 # median is breaking to 2 groups
df1['Decile and Median Spread'] = df1['Decile Return']-df1['Median Return']

# get cumulative return for each decade, and throwing out the other sample years
a=np.cumsum(df1['Decile Return'],axis=0)[1::10] # every 10 years starting from 1 rather than 0, which makes sense
display(a)

# get average yearly retunr for each decade by doing (1940 cumsum - 1930 cumsum)/10
# need to use np.array to drop the index to do array calculation with mismatched index
average_yearly_return_for_each_decade = (np.array(a[1:]) - np.array(a[:-1]))/10

# for better interpretation of result
mydict = {}
for count, item in enumerate(average_yearly_return_for_each_decade):
    mydict['decade before '+str(count*10+1938)]=str(round(item,1))+'%'
mydict



Year
1928     123.56
1938     242.66
1948     377.95
1958     499.55
1968     700.27
1978     917.24
1988    1073.24
1998    1344.63
2008    1475.28
Name: Decile Return, dtype: float64

{'decade before 1938': '11.9%',
 'decade before 1948': '13.5%',
 'decade before 1958': '12.2%',
 'decade before 1968': '20.1%',
 'decade before 1978': '21.7%',
 'decade before 1988': '15.6%',
 'decade before 1998': '27.1%',
 'decade before 2008': '13.1%'}

20th century panics

1920s
Depression of 1920-21, a U.S. economic recession following the end of WW1
Wall Street Crash of 1929 and Great Depression (1929–1939) the worst depression of modern history
1970s
1970s energy crisis
OPEC oil price shock (1973)
1979 energy crisis (1979)
Secondary banking crisis of 1973–1975 in the UK
Latin American debt crisis (late 1970s, early 1980s) known as "lost decade"
1980s
Early 1980s Recession
Chilean crisis of 1982
Bank stock crisis (Israel 1983)
Japanese asset price bubble (1986–1992)
Black Monday (1987) (1987) (US)
Savings and loan crisis failure of 1,043 out of the 3,234 S&Ls from 1986 to 1995 in the U.S.
1990s
Special Period in Cuba (1990–1994)
Early 1990s Recession
1991 India economic crisis
Finnish banking crisis (1990s) (1991-1993)
Swedish banking crisis (1990s)
1994 economic crisis in Mexico
1997 Asian financial crisis
1998 Russian financial crisis
1998-99 Ecuador financial crisis
Argentine economic crisis (1999–2002)
Samba effect (1999) (Brazil)

https://en.wikipedia.org/wiki/List_of_economic_crises

In [4]:
# problem 2 sharpe and treynor ratio

# problem assumed 90 day treasury bond as risk free rate. I assumed it to be an average between 1927 to 2008
# https://fred.stlouisfed.org/series/TB3MS
rfr = 0.055
target = 0

df1['Excess Decile Return'] = df1['Decile Return']-rfr

sharpe_ratio = df1['Excess Decile Return'].mean()/df1['Excess Decile Return'].std()
print('sharpe ratio is '+str(sharpe_ratio))

# Note standard deviation is sqrt of (x-mu)^2/N, if mu is 0 then = return^2.mean().sqrt()
df1.loc[df1['Excess Decile Return']<target,'Excess Downside Return'] = df1['Excess Decile Return']**2
sortino_ratio = (df1['Excess Decile Return'].mean()-rfr)/np.sqrt(df1['Excess Downside Return'].mean())
print('sortino ratio is '+str(sortino_ratio))

df1

sharpe ratio is 0.8507866993000395
sortino ratio is 0.8698190518871524


,Low,2,3,4,5,6,7,8,9,High,High-Low,Decile Return,Quintile Return,Median Return,Decile and Median Spread,Excess Decile Return,Excess Downside Return
Year,,,,,,,,,,,,,,,,,
1927,18.48,9.07,17.17,26.26,32.73,25.11,35.40,27.19,39.44,67.43,48.95,48.95,39.660,18.172,30.778,48.895,NaN
1928,14.72,23.20,21.38,26.09,25.53,34.27,41.87,42.82,48.69,89.33,74.61,74.61,50.050,29.212,45.398,74.555,NaN
1929,-57.59,-47.84,-34.22,-21.34,-2.32,-13.33,5.95,1.32,0.25,-29.87,27.72,27.72,37.905,25.526,2.194,27.665,NaN
1930,-54.58,-48.38,-50.44,-49.55,-31.69,-31.66,-32.42,-19.73,-22.90,-20.87,33.71,33.71,29.595,21.412,12.298,33.655,NaN
1931,-64.51,-58.34,-60.56,-52.91,-53.87,-49.74,-49.39,-42.47,-34.78,-24.23,40.28,40.28,31.920,17.916,22.364,40.225,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2.08,4.53,-1.29,2.10,5.97,7.01,5.97,17.62,11.91,19.34,17.26,17.26,12.320,9.692,7.568,17.205,NaN
2006,22.42,19.48,21.57,14.99,21.31,12.07,12.76,12.08,12.12,3.92,-18.50,-18.50,-12.930,-9.364,-9.136,-18.555,344.288025
2007,-18.83,-8.78,-5.63,-2.81,2.72,6.91,7.70,19.92,9.25,24.79,43.62,43.62,30.825,20.380,23.240,43.565,NaN


In [5]:
#X = df2[['Excess_Market','HML','SMB','UMD']] ## X usually means our input variables (or independent variables)
X = df2['Excess_Market']
#print(X)
y = df1['Excess Decile Return'][:-1] ## Y usually means our output/dependent variable
#print(y)
X = sm.add_constant(X) ## let's add an intercept to our model

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
#predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Excess Decile Return   R-squared:                       0.006
Model:                              OLS   Adj. R-squared:                 -0.007
Method:                   Least Squares   F-statistic:                    0.4469
Date:                  Wed, 18 Nov 2020   Prob (F-statistic):              0.506
Time:                          03:09:19   Log-Likelihood:                -366.10
No. Observations:                    82   AIC:                             736.2
Df Residuals:                        80   BIC:                             741.0
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            18.5109      2.503      7.396      0.000      13.530      23.491
Excess_Market    -0.0753      0.113     -0.669      0.506      -0.299       0.149
==============================================================================
Omnibus:                        6.871   Durbin-Watson:                   1.766
Prob(Omnibus):                  0.032   Jarque-Bera (JB):                7.347
Skew:                          -0.444   Prob(JB):                       0.0254
Kurtosis:                       4.166   Cond. No.                         23.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
treynor_ratio = df1['Excess Decile Return'].mean()/model.params.Excess_Market

print('treynor_ratio is '+str(treynor_ratio))

treynor_ratio is -238.3358868968189


In [7]:
display(pd.merge(df1['Excess Decile Return'],df2[['Excess_Market','HML','SMB','UMD']],on='Year',how='inner').corr())
display(pd.merge(df1['Excess Decile Return'],df2[['Excess_Market','HML','SMB','UMD']],on='Year',how='inner').describe())

,Excess Decile Return,Excess_Market,HML,SMB,UMD
Excess Decile Return,1.000000,-0.074536,-0.284399,-0.235137,0.853023
Excess_Market,-0.074536,1.000000,0.114000,0.412738,-0.028748
HML,-0.284399,0.114000,1.000000,0.075269,-0.304014
SMB,-0.235137,0.412738,0.075269,1.000000,-0.087975
UMD,0.853023,-0.028748,-0.304014,-0.087975,1.000000


,Excess Decile Return,Excess_Market,HML,SMB,UMD
count,82.000000,82.000000,82.000000,82.000000,82.000000
mean,17.936220,7.635732,5.144329,3.562724,10.923963
std,21.211784,21.008801,14.042578,14.367578,12.336426
min,-51.535000,-45.440000,-33.710000,-29.790000,-24.465000
25%,10.377500,-6.620000,-3.838750,-4.730000,4.528750
50%,18.830000,10.180000,5.277500,0.986667,11.860000
75%,30.465000,21.905000,14.208750,11.742500,18.932500
max,74.555000,57.200000,41.085000,54.066667,37.130000
